# Volumes

In order to connect a folder on the host to the container, use the `-v` option of the docker run command. The full syntax is as follows:

```
docker run \
    -v <older on host1>:<folder in container1> \
    -v <older on host2>:<folder in container2> \
    ...
```

By \<folder on hosti\> it can be understood as:

- A path to some folder on the host, this is commonly referred to as **bind mount**, and is used more commonly to pass something specific to that host into the container;
- A **volume**, this is the preferred mount method, and is used to store information that is "created" by the container; it is essentially the same as a folder on the host but controlled by a docker.

# Bind mount

In the following example I create folder `temp_folder`, put it into container named `temp_folder_inc_count`. From container create file there, exited from container and even deleted it, I can get file from host folder.

In [3]:
%%bash
cd filesystem_example
mkdir temp_folder

docker run \
    -v $(pwd)/temp_folder:/temp_folder_in_cont \
    --rm -itd --name temp_example \
    ubuntu &> /dev/null
docker exec temp_example bash -c "echo \'hello from container\' >> temp_folder_in_cont/hello"
docker stop temp_example &> /dev/null

cat temp_folder/hello
rm -r temp_folder

'hello from container'


# Volume

Is a file that resides on the host and doesn't depend on an image, but depends on Docker. Its **primary purpose** is to store data.

### `docker volume` service

`docker volume` is a separate service for managing volumes.

- `ls` - show volumes;
- `create` - create volume;
- `rm` - remove volume;
- `prune` - remove volumes not used in any container;
- `isnpect` - allows you to retrieve information about volume (including where it lies on the host).

The following example shows how to use volumes. Step-by-step description:
- `temp_volume` is created;
- ubuntu container named `example_container` running, and volume created earlier attached to container as `temp_volume_cont` folder;
- file writes to the `temp_volume_cont` from container;
- `example_container` stops and removes automatically (because the `--rm' option was setted);
- next, `inspect` shows where you can find the volume on the host (`Mountpoint` directory), but I can't get access from Jupyter - you should have root privileges for this folder;
- `example_container` was started in the same way as before;
- and in the folder associated with the container, you can still find the file created in the previous steps.

In [1]:
%%bash
docker volume create temp_volume &> /dev/null

docker run \
    -v temp_volume:/temp_volume_cont\
    --rm --name example_container -itd\
    ubuntu &> /dev/null
docker exec example_container bash -c "echo \'hello from volume\' >> temp_volume_cont/hello"
docker stop example_container &> /dev/null

docker volume inspect temp_volume

docker run \
    -v temp_volume:/temp_volume_cont\
    --rm --name example_container -itd\
    ubuntu &> /dev/null
docker exec example_container cat temp_volume_cont/hello
docker stop example_container &> /dev/null

docker volume rm temp_volume &> /dev/null

[
    {
        "CreatedAt": "2023-06-03T19:15:02+03:00",
        "Driver": "local",
        "Labels": null,
        "Mountpoint": "/var/lib/docker/volumes/temp_volume/_data",
        "Name": "temp_volume",
        "Options": null,
        "Scope": "local"
    }
]
'hello from volume'


# Access

### Containers always have root{#sec-containers_always_have_root}

Even if host has root access to some folder/file, **container always works under root**. So if you mount a folder/file in this way, it may lead to unauthorised changes. The following cell contains the example.

In [3]:
%%bash
cd filesystem_example

# creating fodler and file with super secret message
mkdir secret_dir
touch secret_dir/secret_file
echo "super secret info" > secret_dir/secret_file
# Close access to the folder
chmod 000 secret_dir


# make sure we can't access or delte the file
echo "=====From host====="
cat secret_dir/secret_file
rm secret_dir/secret_file


# run container and mount created folder
docker run --rm -itd --name perm_ex\
    -v $(pwd)/secret_dir:/experimental/secret_dir \
    ubuntu &> /dev/null
# and voila ealily extract secret info
echo "=====From docker===="
docker exec perm_ex cat experimental/secret_dir/secret_file
# or even can delete secret file
docker exec perm_ex rm experimental/secret_dir/secret_file
docker exec perm_ex ls experimental/secret_dir

docker stop perm_ex &> /dev/null

=====From host=====


cat: secret_dir/secret_file: Permission denied
rm: cannot remove 'secret_dir/secret_file': Permission denied


=====From docker====
super secret info


### `ro` (read only) option

Continuing from the previous section, note that when you move the folder to the container, you can set the `ro` option, which will prevent the container from modifying the file.

In [1]:
%%bash
echo "some data" > ro_ex
# running container with ro option
docker run --rm -idt --name ro_ex\
    -v $(pwd)/ro_ex:/experimental/ro_ex:ro\
    ubuntu &> /dev/null
# change the file
docker exec ro_ex bash -c "echo \"new some data\" > ro_ex"
# print new file
cat ro_ex

docker stop ro_ex &> /dev/null
rm -r ro_ex

some data


So I tried to change file from container, but even after operation file sill have initial message.

### Runnig with setting user `-u`{sec-run_u_option}

The problem with access can be solved by setting the user when starting the container (`-u` option). Using the example from the section ["Containers always have root"](#sec-containers_always_have_root), you can do this.

In [3]:
%%bash
cd filesystem_example
# создаем папку и в ней файл и даже в него записываем
# сверхсекретное сообщение
mkdir secret_dir
touch secret_dir/secret_file
echo "super secret info" > secret_dir/secret_file
# закрываю доступ в папку
chmod 000 secret_dir

# поднимаем контейнер и монтируем в него данную папку
docker run --rm -itd --name perm_ex -u=1000\
    -v $(pwd)/secret_dir:/experimental/secret_dir \
    ubuntu &> /dev/null
echo "=====Trying to access from a container====="
docker exec perm_ex cat secret_dir/secret_file

docker stop perm_ex &> /dev/null

mkdir: cannot create directory ‘secret_dir’: File exists
touch: cannot touch 'secret_dir/secret_file': Permission denied
bash: line 6: secret_dir/secret_file: Permission denied


=====Trying to access from a container=====


cat: secret_dir/secret_file: No such file or directory


### Mounting `.dockerignore` files

Even if you mount the file described in `.dockerignore`, we will still have it in the container.

In [15]:
%%bash
cd filesystem_example
mkdir app
echo "message in temp_file.txt" > app/temp_file.txt
echo "=====.dockerignore====="
echo "app/temp_file.txt" > .dockerignore
cat .dockerignore

# запускаем контейнер монтируя файл, который указан в .dockerignore
docker run --rm -itd --name ignore_ex\
    -v $(pwd)/app:/app\
    ubuntu &> /dev/null

echo "=====ignore-file из контейнера====="
# убеждаемся, что наш секретный файл преспокойно сидит в контейнере
docker exec ignore_ex cat app/ignore_file.txt

docker stop ignore_ex &> /dev/null
rm -r app
rm .dockerignore

=====.dockerignore=====


cat: .dockerignore: No such file or directory


=====ignore-file из контейнера=====


cat: app/ignore_file.txt: No such file or directory
